## Pasos para la realización del pipeline (AWS & Snowflake)

Importamos las librerías que se usarán

In [ ]:
import json
import math
import numpy as np
import pandas as pd
import snowflake.connector
import time
import boto3
from shapely.geometry import Point, Polygon
import uuid
from datetime import datetime
from decimal import Decimal
from math import radians, sin, cos, sqrt, atan2

Nos conectamos a Snowflake a través del siguiente código de python. Por el tiempo, se procede a usar el usuario de snowflake previamente creado, el cual ya tiene los roles específicos para hacer insert, delete y read.

In [2]:

conn = snowflake.connector.connect (
    user='PRACT3SNOWPYTHON',
    password='8D9$$Q8d6SNaqfk',
    account='TDWJJJC-QD00218',
    warehouse='COMPUTE_WH',
    database='UEV_MU_ADM',
    schema='MODULO_2'
)

Se verifica que los permisos siguen vigentes:

In [3]:
cursor = conn.cursor()
cursor.execute("SHOW GRANTS TO USER PRACT3SNOWPYTHON")
grants = cursor.fetchall()
print("Tus permisos actuales:")
for grant in grants:
    print(grant)

Tus permisos actuales:
(datetime.datetime(2026, 1, 10, 2, 30, 21, 665000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'USAGE', 'ROLE', 'ACCOUNTADMIN', 'ACCOUNTADMIN', 'USER', 'PRACT3SNOWPYTHON', 'false', '')
(datetime.datetime(2026, 1, 13, 1, 4, 14, 300000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'USAGE', 'ROLE', 'PRACT3_ROLE', 'PRACT3_ROLE', 'USER', 'PRACT3SNOWPYTHON', 'false', 'ACCOUNTADMIN')
(datetime.datetime(2026, 1, 10, 4, 38, 38, 342000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'USAGE', 'ROLE', 'QA_ROLE', 'QA_ROLE', 'USER', 'PRACT3SNOWPYTHON', 'false', 'ACCOUNTADMIN')


Verificamos que efectivamente tenemos conexion con alguna tabla ya creada en snowflake, en este caso la C_1_0_0

In [4]:
cursor.execute("SELECT * FROM C_1_0_0 LIMIT 10")

Se procede a crear un dataframe que tome solo las cuatro columnas necesarias para este caso. Especificado en el archivo README.

In [5]:
# Para Valencia
df_vlc = pd.read_csv('Valencia_data.csv', usecols=['id', 'latitude', 'longitude', 'price'])
print(f"Filas cargadas Valencia: {len(df_vlc)}")
print(df_vlc.head())

# Guardar como parquet
df_vlc.to_parquet('Valencia_data.parquet', index=False)
print("✓ Archivo Valencia_data.parquet creado")

Filas cargadas Valencia: 7844
       id  latitude  longitude    price
0   48154  39.48375   -0.37502   $83.00
1  137143  39.36335   -0.31932  $390.00
2  149715  39.46746   -0.32813  $245.00
3  165971  39.46790   -0.38206  $124.00
4  182221  39.46343   -0.34325  $137.00
✓ Archivo Valencia_data.parquet creado


In [6]:
# Para Madrid
df_mad = pd.read_csv('Madrid_data.csv', usecols=['id', 'latitude', 'longitude', 'price'])
print(f"Filas cargadas Madrid: {len(df_mad)}")
print(df_mad.head())

# Guardar como parquet
df_mad.to_parquet('Madrid_data.parquet', index=False)
print("✓ Archivo Madrid_data.parquet creado")

Filas cargadas Madrid: 25000
      id  latitude  longitude    price
0  21853  40.40381   -3.74130      NaN
1  30320  40.41476   -3.70418  $157.00
2  30959  40.41259   -3.70105      NaN
3  40916  40.42247   -3.70577  $143.00
4  62423  40.41884   -3.69655   $65.00
✓ Archivo Madrid_data.parquet creado


In [7]:
# Leer los archivos parquet y verificar que se han guardado correctamente
df_vlc = pd.read_parquet('Valencia_data.parquet')
print(f"Filas cargadas Valencia: {len(df_vlc)}")
print(df_vlc.head())
df_mad = pd.read_parquet('Madrid_data.parquet')
print(f"Filas cargadas Madrid: {len(df_mad)}")
print(df_mad.head())

Filas cargadas Valencia: 7844
       id  latitude  longitude    price
0   48154  39.48375   -0.37502   $83.00
1  137143  39.36335   -0.31932  $390.00
2  149715  39.46746   -0.32813  $245.00
3  165971  39.46790   -0.38206  $124.00
4  182221  39.46343   -0.34325  $137.00
Filas cargadas Madrid: 25000
      id  latitude  longitude    price
0  21853  40.40381   -3.74130     None
1  30320  40.41476   -3.70418  $157.00
2  30959  40.41259   -3.70105     None
3  40916  40.42247   -3.70577  $143.00
4  62423  40.41884   -3.69655   $65.00


------------------------------------------------------------------PASO 2-------------------------------------------------------------------------------

Antes de la realización de este paso, se debe de hacer la adecuación de AWS para que pudiese haber una interconexión entre Glue, Athena, y S3, es decir, hacer un crawler, hacer un database, tener un bucket particionado, que el crawler tenga una política, y que los tres servicios estén bien conectados antes de proceder.

Se procede a la carga de los parquet en athena y procedemos a realizar los siguientes puntos:

-Filtrar por los que tienen precios superiores a 100 dolares en Valencia. Aqui la query en SQL usada:

SELECT 
    id,
    latitude,
    longitude,
    price
FROM valencia
WHERE CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS DOUBLE) > 100
ORDER BY CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS DOUBLE) DESC;

-Seleccionar los 10 primeros id con mayor precio en Valencia. Aqui la query usada:
SELECT 
    id,
    latitude,
    longitude,
    price
FROM valencia
ORDER BY CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS DOUBLE) DESC
LIMIT 10;

-Hacer un join de Valencia y Madrid para ver que ids tienen el mismo precio. Aqui la query usada:
SELECT 
    v.id AS valencia_id,
    m.id AS madrid_id,
    v.price AS shared_price,
    v.latitude AS valencia_lat,
    v.longitude AS valencia_lon,
    m.latitude AS madrid_lat,
    m.longitude AS madrid_lon
FROM valencia v
INNER JOIN madrid m
    ON v.price = m.price
ORDER BY CAST(REPLACE(REPLACE(v.price, '$', ''), ',', '') AS DOUBLE) DESC;

Los resultados de estas queries estarán en el pdf que se entregará con este jupiter notebook

------------------------------------------------------------------PASO 3-------------------------------------------------------------------------------

Siguiendo con el próximo paso, se ingestará la tabla de Valencia que esta en S3 a snowflake. Para ello, nos iremos a snowflake e ingresaremos el siguiente código:


---------------------------------------ACTUALIZAR STORAGE INTEGRATION----------------------------------------------------------

-- Permitimos el database, schema y la ruta correcta en S3, previamente adecuadas con las políticas, usuarios y roles en S3

USE DATABASE UEV_MU_ADM;
USE SCHEMA MODULO_2;
ALTER STORAGE INTEGRATION UEV_MU_ADM_PRACT_1
  SET STORAGE_ALLOWED_LOCATIONS = (
    's3://uev-mu-adm-modulo-2-snow/kaggle/c_1_0_0/1_0_0/'
  );

-- Verificamos
DESC INTEGRATION UEV_MU_ADM_PRACT_1;

---------------------------------------CREAR STAGE----------------------------------------------------------

-- Stage apuntando a la carpeta de Valencia
CREATE OR REPLACE STAGE UEV_MU_ADM.MODULO_2.VALENCIA_STAGE
  STORAGE_INTEGRATION = UEV_MU_ADM_PRACT_1
  URL = 's3://uev-mu-adm-modulo-2-snow/kaggle/c_1_0_0/1_0_0/valencia/';

-- Listar archivos para verificar que está el parquet
LIST @UEV_MU_ADM.MODULO_2.VALENCIA_STAGE;

---------------------------------------CREAR TABLA DESTINO----------------------------------------------------------

CREATE TABLE IF NOT EXISTS UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS (
    ID VARCHAR(100),
    LATITUDE NUMBER(10,6),
    LONGITUDE NUMBER(10,6),
    PRICE NUMBER(10,2),
    LOAD_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

---------------------------------------CREAR FILE FORMAT----------------------------------------------------------

CREATE OR REPLACE FILE FORMAT UEV_MU_ADM.MODULO_2.PARQUET_FORMAT
  TYPE = 'PARQUET';

---------------------------------------COPIAR DATOS A LA TABLA----------------------------------------------------------

-- Cargar el archivo parquet en la tabla
COPY INTO UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS (ID, LATITUDE, LONGITUDE, PRICE)
FROM (
  SELECT 
    $1:id::VARCHAR,
    $1:latitude::NUMBER(10,6),
    $1:longitude::NUMBER(10,6),
    TO_NUMBER(REPLACE(REPLACE($1:price::VARCHAR, '$', ''), ',', ''), 10, 2)
  FROM @UEV_MU_ADM.MODULO_2.VALENCIA_STAGE/FileYear=2026/FileMonth=01/FileDay=16/Valencia_data.parquet
)
FILE_FORMAT = UEV_MU_ADM.MODULO_2.PARQUET_FORMAT;

---------------------------------------VERIFICAR DATOS CARGADOS----------------------------------------------------------

-- Ver cuántas filas se cargaron
SELECT COUNT(*) FROM UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS;

-- Ver los primeros 10 registros
SELECT * FROM UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS 
ORDER BY LOAD_TIMESTAMP DESC 
LIMIT 10;


Antes de realizar este paso se deben de especificar las políticas, el trust relationship y los roles con usuarios en AWS.

------------------------------------------------------------------PASO 4-------------------------------------------------------------------------------

Para el paso 4, creacion de una transient table de los datos en una tarea para quedarnos solo con el 90% de los apartamentos con los precios más altos, iremos de igual forma a snowflake e ingresaremos lo siguiente:




---------------------------------------CREAR TRANSIENT TABLE----------------------------------------------------------

-- Crear tabla transient para almacenar el top 90% de precios más altos
CREATE OR REPLACE TRANSIENT TABLE UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES (
    ID VARCHAR(100),
    LATITUDE NUMBER(10,6),
    LONGITUDE NUMBER(10,6),
    PRICE NUMBER(10,2),
    PERCENTILE_RANK NUMBER(5,2),
    LOAD_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

---------------------------------------CREAR STORED PROCEDURE----------------------------------------------------------

-- Procedimiento para calcular y cargar el top 90%
CREATE OR REPLACE PROCEDURE UEV_MU_ADM.MODULO_2.LOAD_TOP_90_PRICES()
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Truncar la tabla antes de insertar datos nuevos
    TRUNCATE TABLE UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES;
    
    -- Insertar el 90% de apartamentos con precios más altos
    INSERT INTO UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES (ID, LATITUDE, LONGITUDE, PRICE, PERCENTILE_RANK)
    SELECT 
        ID,
        LATITUDE,
        LONGITUDE,
        PRICE,
        PERCENT_RANK() OVER (ORDER BY PRICE DESC) * 100 AS PERCENTILE_RANK
    FROM UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS
    QUALIFY PERCENT_RANK() OVER (ORDER BY PRICE DESC) <= 0.90;
    
    RETURN 'Top 90% de precios cargado exitosamente';
END;
$$;

---------------------------------------CREAR TAREA (TASK)----------------------------------------------------------

-- Crear warehouse si no existe (necesario para ejecutar tasks)
CREATE WAREHOUSE IF NOT EXISTS COMPUTE_WH
  WITH WAREHOUSE_SIZE = 'X-SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

-- Crear tarea que se ejecuta diariamente a las 2 AM
CREATE OR REPLACE TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES
  WAREHOUSE = COMPUTE_WH
  SCHEDULE = 'USING CRON 0 2 * * * UTC'  -- Todos los días a las 2 AM UTC
  AS
  CALL UEV_MU_ADM.MODULO_2.LOAD_TOP_90_PRICES();

-- Activar la tarea
ALTER TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES RESUME;

---------------------------------------EJECUTAR MANUALMENTE (PRIMERA VEZ)----------------------------------------------------------

-- Ejecutar el procedimiento manualmente la primera vez
CALL UEV_MU_ADM.MODULO_2.LOAD_TOP_90_PRICES();

---------------------------------------VERIFICAR RESULTADOS----------------------------------------------------------

-- Ver cuántos registros se cargaron (debe ser ~90% del total)
SELECT COUNT(*) as total_top_90 FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES;

-- Comparar con el total original
SELECT 
    (SELECT COUNT(*) FROM UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS) as total_original,
    (SELECT COUNT(*) FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES) as total_top_90,
    ROUND((SELECT COUNT(*) FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES) * 100.0 / 
          (SELECT COUNT(*) FROM UEV_MU_ADM.MODULO_2.VALENCIA_LISTINGS), 2) as porcentaje;

-- Ver los precios mínimo y máximo del top 90%
SELECT 
    MIN(PRICE) as precio_minimo_top90,
    MAX(PRICE) as precio_maximo_top90,
    AVG(PRICE) as precio_promedio_top90
FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES;

-- Ver los primeros 10 registros
SELECT * FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES
ORDER BY PRICE DESC
LIMIT 10;

---------------------------------------MONITOREAR LA TAREA----------------------------------------------------------

-- Ver el historial de ejecución de la tarea
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
    TASK_NAME => 'TASK_LOAD_TOP_90_PRICES',
    SCHEDULED_TIME_RANGE_START => DATEADD('day', -7, CURRENT_TIMESTAMP())
))
ORDER BY SCHEDULED_TIME DESC;

-- Ver el estado de la tarea
SHOW TASKS LIKE 'TASK_LOAD_TOP_90_PRICES' IN SCHEMA UEV_MU_ADM.MODULO_2;

-- Ejecutar la tarea manualmente (sin esperar al schedule)
EXECUTE TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES;

---------------------------------------PAUSAR/REANUDAR/TERMINAR LA TAREA----------------------------------------------------------

-- Pausar la tarea
ALTER TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES SUSPEND;

-- Reanudar la tarea
ALTER TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES RESUME;

-- Tumbar la tarea
DROP TASK UEV_MU_ADM.MODULO_2.TASK_LOAD_TOP_90_PRICES;


Para automatizar la tarea, incorporar un cron que cada cierto tiempo la active o al procedimiento opcional que se uso. Ejemplo: 
-- Cada hora
SCHEDULE = 'USING CRON 0 * * * * UTC'

------------------------------------------------------------------PASO 5-------------------------------------------------------------------------------

Para la ingesta del geojson de valencia en snowflake, se realizo lo siguiente:

In [8]:
create_table_query_variant = """
CREATE TABLE IF NOT EXISTS UEV_MU_ADM.MODULO_2.GEOJSON_WITH_VARIANT (
    ID INT AUTOINCREMENT,
    CITY VARCHAR(100) NOT NULL,
    GEOJSON_DATA VARIANT,  -- Columna VARIANT para almacenar JSON completo
    LOAD_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
)
"""

cursor.execute(create_table_query_variant)
print("✅ Tabla GEOJSON_WITH_VARIANT creada")

# Leer el archivo GeoJSON
with open('Valencia_geojson.json', 'r') as f:
    geojson_data = json.load(f)

# Insertar el GeoJSON completo en la columna VARIANT
insert_variant_query = """
INSERT INTO UEV_MU_ADM.MODULO_2.GEOJSON_WITH_VARIANT (CITY, GEOJSON_DATA)
SELECT 'Valencia', PARSE_JSON(%s)
"""

cursor.execute(insert_variant_query, (json.dumps(geojson_data),))
conn.commit()
print("✅ GeoJSON completo insertado")

# Consultar y extraer datos del VARIANT
cursor.execute("""
SELECT 
    CITY,
    GEOJSON_DATA:type::STRING as geometry_type,
    GEOJSON_DATA:features[0]:geometry:coordinates[0] as coordinates
FROM UEV_MU_ADM.MODULO_2.GEOJSON_WITH_VARIANT
""")

for row in cursor:
    print(row)

✅ Tabla GEOJSON_WITH_VARIANT creada
✅ GeoJSON completo insertado
('Valencia', 'FeatureCollection', '[\n  [\n    -0.37093004277016917,\n    39.47664277831467\n  ],\n  [\n    -0.3753860808744207,\n    39.479222421744\n  ],\n  [\n    -0.37888723908901056,\n    39.48075783178862\n  ],\n  [\n    -0.3840597244534081,\n    39.48011312643061\n  ],\n  [\n    -0.3871626530358583,\n    39.478454640403044\n  ],\n  [\n    -0.390226561200393,\n    39.47747202511246\n  ],\n  [\n    -0.3888334463088654,\n    39.47394023652714\n  ],\n  [\n    -0.38628724389192826,\n    39.471882574627756\n  ],\n  [\n    -0.38437754229090615,\n    39.469118454509584\n  ],\n  [\n    -0.38318395190526644,\n    39.467183522452046\n  ],\n  [\n    -0.38246796887293044,\n    39.46491052053989\n  ],\n  [\n    -0.3808367392757077,\n    39.463497607593695\n  ],\n  [\n    -0.37805174582172185,\n    39.463159684893895\n  ],\n  [\n    -0.3739935579543783,\n    39.46398905109291\n  ],\n  [\n    -0.3705321703033633,\n    39.465371220

------------------------------------------------------------------PASO 6-------------------------------------------------------------------------------

Se realizó el siguiente código:

In [ ]:


# ===================== FUNCIÓN PARA CALCULAR DISTANCIA (Haversine) =====================
def calcular_distancia_km(lat1, lon1, lat2, lon2):
    """
    Calcula la distancia en kilómetros entre dos puntos usando la fórmula de Haversine
    """
    R = 6371  # Radio de la Tierra en kilómetros
    
    lat1_rad = radians(lat1)
    lat2_rad = radians(lat2)
    delta_lat = radians(lat2 - lat1)
    delta_lon = radians(lon2 - lon1)
    
    a = sin(delta_lat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(delta_lon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distancia = R * c
    return round(distancia, 2)

# ===================== CENTRO DE VALENCIA =====================
# Coordenadas aproximadas del centro de Valencia (Plaza del Ayuntamiento)
CENTRO_VALENCIA_LAT = 39.4699
CENTRO_VALENCIA_LON = -0.3763

print(f"📍 Centro de Valencia: ({CENTRO_VALENCIA_LAT}, {CENTRO_VALENCIA_LON})")

# ===================== LEER TRANSIENT TABLE DE VALENCIA =====================
query_valencia = """
SELECT 
    ID,
    LATITUDE,
    LONGITUDE,
    PRICE,
    PERCENTILE_RANK
FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES
"""

cursor.execute(query_valencia)
df_valencia = cursor.fetch_pandas_all()
print(f"\n✅ Apartamentos cargados desde Snowflake: {len(df_valencia)}")

# ===================== LEER DATOS DEL GEOJSON =====================
query_geojson = """
SELECT 
    ORDEN,
    CITY,
    LATITUDE,
    LONGITUDE
FROM UEV_MU_ADM.MODULO_2.GEOJSON
ORDER BY ORDEN
"""

cursor.execute(query_geojson)
df_geojson = cursor.fetch_pandas_all()
print(f"\n✅ Puntos del polígono GeoJSON: {len(df_geojson)}")
print(df_geojson.head())

# ===================== CREAR POLÍGONO DE VALENCIA =====================
# Convertir coordenadas del GeoJSON a un polígono de Shapely
coordenadas_poligono = [(row['LONGITUDE'], row['LATITUDE']) for _, row in df_geojson.iterrows()]
poligono_valencia = Polygon(coordenadas_poligono)

print(f"\n🗺️ Polígono creado con {len(coordenadas_poligono)} puntos")
print(f"Área del polígono: {poligono_valencia.area:.6f} grados cuadrados")

# ===================== FILTRAR APARTAMENTOS DENTRO DEL ÁREA =====================
apartamentos_dentro = []

for _, row in df_valencia.iterrows():
    punto = Point(row['LONGITUDE'], row['LATITUDE'])
    
    if poligono_valencia.contains(punto):
        # Calcular distancia al centro de Valencia
        distancia = calcular_distancia_km(
            row['LATITUDE'], 
            row['LONGITUDE'],
            CENTRO_VALENCIA_LAT,
            CENTRO_VALENCIA_LON
        )
        
        apartamentos_dentro.append({
            'ID': row['ID'],
            'LATITUDE': row['LATITUDE'],
            'LONGITUDE': row['LONGITUDE'],
            'PRICE': row['PRICE'],
            'PERCENTILE_RANK': row['PERCENTILE_RANK'],
            'DISTANCIA_AL_CENTRO_KM': distancia,
            'DENTRO_DEL_AREA': True
        })

# ===================== CREAR DATAFRAME FINAL =====================
df_apartamentos_filtrados = pd.DataFrame(apartamentos_dentro)
df_apartamentos_filtrados = df_apartamentos_filtrados.dropna(
    subset=[
        'LATITUDE',
        'LONGITUDE',
        'PRICE',
        'PERCENTILE_RANK',
        'DISTANCIA_AL_CENTRO_KM'
    ]
)

print(f"\n🏠 Apartamentos dentro del área de Valencia: {len(df_apartamentos_filtrados)}")
print(f"📊 Porcentaje dentro del área: {len(df_apartamentos_filtrados) / len(df_valencia) * 100:.2f}%")

# ===================== ESTADÍSTICAS =====================
if len(df_apartamentos_filtrados) > 0:
    print("\n📈 ESTADÍSTICAS:")
    print(f"Distancia mínima al centro: {df_apartamentos_filtrados['DISTANCIA_AL_CENTRO_KM'].min():.2f} km")
    print(f"Distancia máxima al centro: {df_apartamentos_filtrados['DISTANCIA_AL_CENTRO_KM'].max():.2f} km")
    print(f"Distancia promedio al centro: {df_apartamentos_filtrados['DISTANCIA_AL_CENTRO_KM'].mean():.2f} km")
    print(f"Precio promedio: ${df_apartamentos_filtrados['PRICE'].mean():.2f}")
    
    print("\n🏆 Top 10 apartamentos más cercanos al centro:")
    print(df_apartamentos_filtrados.nsmallest(10, 'DISTANCIA_AL_CENTRO_KM')[
        ['ID', 'PRICE', 'DISTANCIA_AL_CENTRO_KM', 'LATITUDE', 'LONGITUDE']
    ])
    
    print("\n💰 Top 10 apartamentos más caros dentro del área:")
    print(df_apartamentos_filtrados.nlargest(10, 'PRICE')[
        ['ID', 'PRICE', 'DISTANCIA_AL_CENTRO_KM', 'LATITUDE', 'LONGITUDE']
    ])
else:
    print("\n⚠️ No se encontraron apartamentos dentro del área definida")


📍 Centro de Valencia: (39.4699, -0.3763)

✅ Apartamentos cargados desde Snowflake: 7072

✅ Puntos del polígono GeoJSON: 30
   ORDEN      CITY   LATITUDE  LONGITUDE
0      0    Madrid  40.424564  -3.711566
1      0  Valencia  39.476643  -0.370930
2      1    Madrid  40.422141  -3.714362
3      1  Valencia  39.479222  -0.375386
4      2    Madrid  40.417244  -3.712703

🗺️ Polígono creado con 30 puntos
Área del polígono: 0.018114 grados cuadrados

🏠 Apartamentos dentro del área de Valencia: 1563
📊 Porcentaje dentro del área: 22.10%

📈 ESTADÍSTICAS:
Distancia mínima al centro: 0.03 km
Distancia máxima al centro: 3.26 km
Distancia promedio al centro: 0.76 km
Precio promedio: $153.95

🏆 Top 10 apartamentos más cercanos al centro:
                      ID   PRICE  DISTANCIA_AL_CENTRO_KM   LATITUDE  LONGITUDE
665  1512890541606203698   168.0                    0.03  39.470011  -0.376613
268             48823848  1256.0                    0.05  39.470350  -0.376530
306             37142370   35

------------------------------------------------------------------PASO 7-------------------------------------------------------------------------------

In [16]:
# ===================== GUARDAR RESULTADOS EN SNOWFLAKE=====================

#Insertar de vuelta a Snowflake
create_filtered_table = """
CREATE OR REPLACE TABLE UEV_MU_ADM.MODULO_2.VALENCIA_FILTERED_BY_AREA (
    ID VARCHAR(100),
    LATITUDE NUMBER(10,6),
    LONGITUDE NUMBER(10,6),
    PRICE NUMBER(10,2),
    PERCENTILE_RANK NUMBER(5,2),
    DISTANCIA_AL_CENTRO_KM NUMBER(10,2),
    DENTRO_DEL_AREA BOOLEAN,
    LOAD_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
)
"""

cursor.execute(create_filtered_table)
print("\n✅ Tabla VALENCIA_FILTERED_BY_AREA creada en Snowflake")

# Insertar datos
data_to_insert = [
    (
        row['ID'],
        float(row['LATITUDE']),
        float(row['LONGITUDE']),
        float(row['PRICE']),
        float(row['PERCENTILE_RANK']),
        float(row['DISTANCIA_AL_CENTRO_KM']),
        True
    )
    for _, row in df_apartamentos_filtrados.iterrows()
]

insert_query = """
INSERT INTO UEV_MU_ADM.MODULO_2.VALENCIA_FILTERED_BY_AREA 
(ID, LATITUDE, LONGITUDE, PRICE, PERCENTILE_RANK, DISTANCIA_AL_CENTRO_KM, DENTRO_DEL_AREA)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_query, data_to_insert)
conn.commit()
print(f"✅ {len(data_to_insert)} apartamentos insertados en Snowflake")

# ===================== VISUALIZAR DATAFRAME FINAL =====================
print("\n📋 DATAFRAME FINAL (primeras 20 filas):")
print(df_apartamentos_filtrados.head(20))



✅ Tabla VALENCIA_FILTERED_BY_AREA creada en Snowflake
✅ 1563 apartamentos insertados en Snowflake

📋 DATAFRAME FINAL (primeras 20 filas):
                      ID   LATITUDE  LONGITUDE    PRICE  PERCENTILE_RANK  \
266  1484165221264769599  39.474504  -0.384803  10000.0            11.05   
267  1264452543403641618  39.471844  -0.394216   9143.0            11.17   
268             48823848  39.470350  -0.376530   1256.0            11.76   
269             25360723  39.467160  -0.368720   1252.0            11.77   
270             48837508  39.470100  -0.374850   1215.0            11.81   
271  1420861031773004759  39.465469  -0.376300   1000.0            11.83   
272   783233881834500810  39.472845  -0.384255    765.0            11.93   
273   812051449237491927  39.472642  -0.379957    742.0            12.00   
274   597542675554668322  39.473732  -0.372222    689.0            12.04   
275             51860040  39.474630  -0.374320    665.0            12.06   
276             13430117 

Se procedió a subir esta tabla final de snowflake a S3, para ello, nos dirigimos a snowflake y colocamos el siguiente código:



COPY INTO @UEV_MU_ADM.MODULO_2.VALENCIA_STAGE
FROM (
    SELECT
        ID,
        LATITUDE,
        LONGITUDE,
        PRICE,
        PERCENTILE_RANK,
        LOAD_TIMESTAMP,
        'FileYear=' || YEAR(CURRENT_DATE()) ||
        '/FileMonth=' || LPAD(MONTH(CURRENT_DATE()), 2, '0') ||
        '/FileDay=' || LPAD(DAY(CURRENT_DATE()), 2, '0') AS partition_path
    FROM UEV_MU_ADM.MODULO_2.VALENCIA_TOP_90_PRICES
)
PARTITION BY (partition_path)
FILE_FORMAT = UEV_MU_ADM.MODULO_2.PARQUET_FORMAT



--Se verifica si se ingreso sin problemas
LIST @UEV_MU_ADM.MODULO_2.VALENCIA_STAGE;

------------------------------------------------------------------PASO 8-------------------------------------------------------------------------------

Para poder hacer el paso en DynamoDB deberemos instalar primero la dependencia necesaria en la terminal, es decir, pip install boto3 geojson. Tras esto, creamos la tabla

In [ ]:
dynamodb = boto3.resource('dynamodb', region_name='eu-west-1')

table = dynamodb.create_table(
    TableName='Valencia_GeoJSON',
    KeySchema=[
        {'AttributeName': 'pk', 'KeyType': 'HASH'},
        {'AttributeName': 'sk', 'KeyType': 'RANGE'}
    ],
    AttributeDefinitions=[
        {'AttributeName': 'pk', 'AttributeType': 'S'},
        {'AttributeName': 'sk', 'AttributeType': 'S'}
    ],
    BillingMode='PAY_PER_REQUEST'
)

table.wait_until_exists()
print("✅ Tabla creada")


✅ Tabla creada


In [ ]:


def float_to_decimal(obj):
    """
    Convierte recursivamente floats a Decimal
    (necesario para DynamoDB)
    """
    if isinstance(obj, float):
        return Decimal(str(obj))
    elif isinstance(obj, list):
        return [float_to_decimal(i) for i in obj]
    elif isinstance(obj, dict):
        return {k: float_to_decimal(v) for k, v in obj.items()}
    else:
        return obj

# ===================== CONFIG =====================
TABLE_NAME = 'Valencia_GeoJSON'
REGION = 'eu-west-1'
GEOJSON_FILE = 'Valencia_geojson.json'

# ===================== CONEXIÓN =====================
dynamodb = boto3.resource('dynamodb', region_name=REGION)
table = dynamodb.Table(TABLE_NAME)

# ===================== LEER GEOJSON =====================
with open(GEOJSON_FILE, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

features = geojson_data.get('features', [])
print(f"📍 Features encontrados: {len(features)}")

# ===================== INGESTA =====================
# Función auxiliar para convertir floats a Decimal recursivamente
def dict_to_decimal(obj):
    if isinstance(obj, list):
        return [dict_to_decimal(i) for i in obj]
    elif isinstance(obj, dict):
        return {k: dict_to_decimal(v) for k, v in obj.items()}
    elif isinstance(obj, float):
        # Usamos str(obj) para evitar errores de precisión de punto flotante
        return Decimal(str(obj))
    return obj

# Tu lógica de ingesta corregida
with table.batch_writer() as batch:
    for feature in features:
        feature_id = str(uuid.uuid4())

        # Convertimos TODA la estructura a Decimal
        geometry = dict_to_decimal(feature.get('geometry', {}))
        properties = dict_to_decimal(feature.get('properties', {}))

        item = {
            'pk': feature_id,
            'sk': geometry.get('type', 'UNKNOWN'),
            'city': properties.get('city', 'Valencia'),
            'geometry': geometry,
            'properties': properties,
            'created_at': datetime.utcnow().isoformat()
        }

        batch.put_item(Item=item)

print("✅ GeoJSON cargado correctamente con tipos Decimal")

📍 Features encontrados: 1


C:\Users\fer_p\AppData\Local\Temp\ipykernel_41508\2827768705.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'created_at': datetime.utcnow().isoformat()


✅ GeoJSON cargado correctamente con tipos Decimal


# Comentarios Adicionales
La evidencia del proceso de creación estará estipulado en el pdf llamado results.